In [1]:
import pandas as pd
import numpy as np
import ast 
import json
import time

In [2]:
electronics_df = pd.read_json('../reviews_Electronics_5.json', lines=True)
#apps_df = pd.read_json('../reviews_Apps_for_Android_5.json', lines=True)
cds_df = pd.read_json('../reviews_CDs_and_Vinyl_5.json', lines=True)
#kindle_df = pd.read_json('../reviews_Kindle_Store_5.json', lines=True)
#movies_tv_df = pd.read_json('../reviews_Movies_and_TV_5.json', lines=True)


In [3]:
# 1262304000 is unix time for January 1, 2010

electronics_df_cut = electronics_df[electronics_df['unixReviewTime'] > 1262304000]
#apps_df_cut = apps_df[apps_df['unixReviewTime'] > 1262304000]
cds_df_cut = cds_df[cds_df['unixReviewTime'] > 1262304000]
#kindle_df_cut = kindle_df[kindle_df['unixReviewTime'] > 1262304000]
#movies_tv_df_cut = movies_tv_df[movies_tv_df['unixReviewTime'] > 1262304000]



In [4]:
electronics_df_cut['category'] = 'Electronics'
#apps_df_cut['category'] = 'Apps for Android'
cds_df_cut['category'] = 'CDs and Vinyl'
#kindle_df_cut['category'] = 'Kindle Store'
#movies_tv_df_cut['category'] = 'Movies and TV'

/Users/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/david/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
dataframes = [electronics_df_cut, cds_df_cut]

reviews_df = pd.concat(dataframes)

In [6]:
reviews_df = reviews_df.reset_index(drop=True)

In [7]:
reviews_df.to_csv('reviews (no books).csv', index=False)

In [ ]:
unique_asin = set(reviews_df['asin'])

unique_asin_list = pd.Series(list(unique_asin))

unique_asin_list.to_csv('unique_asin.csv')

In [ ]:
del electronics_df
#del apps_df
del cds_df
#del kindle_df
#del movies_tv_df

del electronics_df_cut
#del apps_df_cut
del cds_df_cut
#del kindle_df_cut
#del movies_tv_df_cut


In [ ]:
# Parses the meta data

category = 'Books'

path = ('../meta_' + category + '.json')

# Start a pandas df to capture what we need from the metadata
product_df = pd.DataFrame({})
product_df['ASIN']=''
product_df['Title']=''
product_df['Sale Rank']=''

start = time.time()
total_logged =0
logged = 0
no_title = 0
no_sales_rank = 0

file_no = 1

# open the json file with the ast modules so we can capture the values 
with open(path) as file:
    for j,line in enumerate(file):
        row = ast.literal_eval(line)
        
        if row['asin'] in unique_book_asin:
            total_logged += 1
            logged += 1
            product_df.at[j,'ASIN'] = row['asin']
            
            try:
                product_df.at[j,'Title'] = row['title']
            except:
                product_df.at[j,'Title'] = np.nan
                no_title += 1
            
            try:
                product_df.at[j,'Sale Rank'] = list(row['salesRank'].values())[0]
            except:
                product_df.at[j,'Sale Rank'] = np.nan
                no_sales_rank += 1
        if j > 0 and j % 10000 == 0:
            time_elapsed = time.time() - start
            print(f'Reach {j} rows of file. Logged for next file {logged}. Total log: {total_logged}. No title: {no_title}. No sales: {no_sales_rank}. Time elapsed: {time_elapsed}')
            start = time.time()
        
        if logged == 10000:
            
            logged = 0
            product_df.set_index('ASIN', inplace=True)
            product_df.to_csv('meta_' + category + '_file' + str(file_no).zfill(4) + '.csv')
            file_no += 1
            del product_df
            product_df = pd.DataFrame({})
            product_df['ASIN']=''
            product_df['Title']=''
            product_df['Sale Rank']=''
        

print('Done.')

#product_df = product_df.dropna(axis=0,how='any',subset=['ASIN'])
#product_df = product_df.drop_duplicates(subset=['ASIN','Title'],keep='last')

#product_df = product_df.set_index('ASIN')  
#export to csv
#print('Exporting to csv...')

#output = 'Metadata Extract.csv'
#product_df.to_csv(output)

In [ ]:
kindle1 = pd.read_csv('meta_Kindle_Store_file0001')

In [ ]:
kindle1

In [ ]:
reviews_df.columns

In [ ]:
#reads the Books json, filters data, produces smaller csv files for iterative loading

# path

path = ('../../reviews_Books_5.json')

# Start a pandas df to capture what we need from the metadata
bookreviews_df = pd.DataFrame({})
bookreviews_df['asin'] = ''
bookreviews_df['helpful'] = ''
bookreviews_df['overall'] = ''
bookreviews_df['reviewText'] = ''
bookreviews_df['reviewTime'] = ''
''


#'asin', 'category', 'helpful', 'overall', 'reviewText', 'reviewTime','reviewerID', 'reviewerName', 'summary', 'unixReviewTime'

start = time.time()
loop_start = time.time()
total_logged =0
logged = 0
no_title = 0
no_sales_rank = 0

file_no = 1

# open the json file with the ast modules so we can capture the values 
with open(path) as file:
    for j,line in enumerate(file):
        row = ast.literal_eval(line)
        
        if int(row['reviewTime'][-4:]) > 2009:

            total_logged += 1
            logged += 1

            try:
                bookreviews_df.at[j,'asin'] = row['asin']
            except:
                continue

            try:
                bookreviews_df.at[j,'helpful'] = row['helpful']
            except:
                bookreviews_df.at[j,'helpful'] = np.nan

            try:
                bookreviews_df.at[j,'overall'] = row['overall']
            except:
                bookreviews_df.at[j,'overall'] = np.nan

            try:
                bookreviews_df.at[j,'reviewText'] = row['reviewText']
            except:
                bookreviews_df.at[j,'reviewText'] = np.nan
            
            try:
                bookreviews_df.at[j,'reviewTime'] = row['reviewTime']
            except:
                bookreviews_df.at[j,'reviewTime'] = np.nan        
    
        if j > 0 and j % 100000 == 0:
            time_elapsed = time.time() - loop_start
            print(f'Reached {j} rows of file. Total logged: {total_logged}. Time elapsed: {time_elapsed}')
            loop_start = time.time()
        
        if logged == 1000:
            
            logged = 0
            bookreviews_df.set_index('asin', inplace=True)
            bookreviews_df.to_csv('reviews_books_' + str(file_no).zfill(4) + '.csv')
            file_no += 1
            del bookreviews_df
            bookreviews_df = pd.DataFrame({})
            bookreviews_df['asin'] = ''
            bookreviews_df['helpful'] = ''
            bookreviews_df['overall'] = ''
            bookreviews_df['reviewText'] = ''
            bookreviews_df['reviewTime'] = ''
        
        #stops after 3 million reviews
        if logged == 3000000:
            break

print('Done.')

In [ ]:
file_names = []

for file_no in range(2, 7140):

    filepath = 'Book Reviews/reviews_books_' + str(file_no).zfill(4) + '.csv'
    file_names.append(filepath)


In [ ]:
book_review_df = pd.read_csv('Book Reviews/reviews_books_0001.csv')

books_asin = []

i = 0

for file in file_names:
    i += 1
    
    book_review_append = pd.read_csv(file)
    book_review_df = pd.concat([book_review_df, book_review_append])
    book_review_df.reset_index(drop=True) 

    if i % 100 == 0:
        print(file)
    
    if i == 100:
        i = 0
        books_asin.append(book_review_df['asin'])
        book_review_df = pd.read_csv(file)
    
books_unique_asin = set(books_asin)


In [ ]:
total = []

for list in books_asin:
    
    add = list.tolist()
    
    for asin in add:
        total.append(asin)

In [ ]:
unique_book_asin = set(total)